# Generate a sample frame for creating image pairs


In [ ]:
# Read Video

import cv2
import os
from cv2 import __version__
__version__
import random
from tqdm import tqdm
import pandas as pd
# folder = "../_data/_raw/videos/Box11"

from tqdm import tqdm

In [ ]:
import glob
videofolder = "../../_data/00_raw/videos_current_highres/"
sample_frame_folder  = "../../_data/02_siteplan/sample_frames/current_sample"
exportfolder = "../../_data/03_tracking_result/_current_attr_result"
predictfolder = "../../_data/03_tracking_result/_current_attr_result/_raw_prediction"

# Steps
1. Create a list of all the videos
2. Annotate if they have been processed or not
3. Create a folder for sample image starting time
4. Create a list for all scenes partitioned
5. Get a sample image whenever the camera moved
6. Convert the raw prediction to combined attribute file

In [ ]:

# list all videos in the folder
videols = glob.glob(videofolder + "**/*.avi", 
                    recursive = True)

print(f"Total videos {len(videols) = }")

videopath = pd.DataFrame({
    "videopath": videols,
    "video_name": [x.split("\\")[-1] for x in videols],
    "video_location": [x.split("\\")[-2] for x in videols],
})
videopath["video_location"].unique()

video_location = [
    'bryant_park',
    'Chestnut Street videos (NEW)',
    'Downtown Crossing videos (NEW)',
    'Met Steps videos (NEW)'
       ]
for i, path in enumerate(videopath.videopath.values):
    for loc in video_location:
        if loc in path:
            videopath.loc[i, "video_location"] = loc
            
videopath = videopath[videopath["video_location"].isin(video_location)]
print(f"Total videos {videopath.shape[0] = }")

In [ ]:
# export the video path
# videopath['video_indicator'] = "new_video"
# videopath.to_csv("../_data/00_raw/_video_meta/video_path.csv", 
#                  index = False)

# 2. Pick the ones ready to generate sample frames

In [ ]:
video_sample = {
    'bryant_park':["20081008-072238b01.avi","20081008-072238b18.avi","20081008-141944b01.avi"],
    'Chestnut Street videos (NEW)':["20100519-083343b04.avi"],
    'Downtown Crossing videos (NEW)':["20100521-074701b06.avi","20100521-115755b02.avi"],
    'Met Steps videos (NEW)':["20100612-120118b01.avi","20100612-082221b01.avi","20100612-082221b02.avi","20100612-082221b03.avi"]
}

allvideos = []
for v in video_sample.values():
    allvideos.extend(v)
allvideos

In [ ]:
# load the videopath again to retrieve starting and end time
videopath = pd.read_csv("../../_data/00_raw/_video_meta/video_path_0509.csv")
videopath['video_id'] = videopath['video_name'].apply(lambda x: x.split(".")[0])
# Generate sample frames from the list of videos
videopath['videopath'] = videopath['videopath'].apply(lambda x: x.replace("../", "../../"))
videosel = videopath[videopath["video_name"].isin(allvideos)].reset_index(drop = True)
# videosel.shape

In [ ]:
# read the video to get their fps and frame count
from tqdm import tqdm
def getbasics(file_path):
    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    # print('frames per second =',fps)
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    # print('frames size =',size)
    # video.release()
    return video, fps, size, length

for i, path in enumerate(videosel.videopath.values):
    video, fps, size, length = getbasics(path)
    videosel.loc[i, "fps"] = fps
    videosel.loc[i, "w"] = size[0]
    videosel.loc[i, "h"] = size[1]
    videosel.loc[i, "length"] = length
    
    name = videosel.loc[i, "video_name"].split(".")[0]
    # set the video to the first frame and export a sample image
    # this is a generic selection. After split all video to scenes, reselect the first frame dynamically
    second_start = 55
    minute_start = 1
    first_frame = int(fps*(minute_start*60 + second_start))
    video.set(cv2.CAP_PROP_POS_FRAMES, first_frame)
    success = True
    while success:     
        success, image = video.read()
        try:
            cv2.imwrite(os.path.join(sample_frame_folder, f"{name}_{first_frame}.jpg"), image)
            # save frame as JPEG file 
            break
        except:
            first_frame += 1
            print("first frame not working")

In [ ]:


# integrate the start and end timestamp
videopath = pd.read_csv("../../_data/00_raw/_video_meta/video_path_0509.csv")
videopath['video_id'] = videopath['video_name'].apply(lambda x: x.split(".")[0])
videopath = videopath[~videopath['video_name'].str.contains("CORRUPT")].reset_index(drop = True)
videopath['video_group'] = videopath['video_id'].apply(lambda x: x.split("b")[0])
videopath['video_section'] = videopath['video_id'].apply(lambda x: int(x.split("b")[1]))
# load finished prediction names
exportfolder = "../../_data/03_tracking_result/_current_attr_result"
finished = os.listdir(exportfolder)
videopath['finished'] = videopath['video_id'].apply(lambda x: x+".csv" in finished)

videopath['video_group_started_at'] = pd.to_datetime(videopath['video_group'])

videopath['video_length_s'] = videopath['length']/29.97
# convert the column to timeDelta
videopath['video_length_s'] = videopath['video_length_s'].apply(lambda x: pd.Timedelta(seconds = x))

videopath['video_section_started_at'] = videopath['video_group_started_at']+ \
    (videopath['video_section']-1)*videopath['video_length_s']
# videopath.to_csv("../../_data/00_raw/_video_meta/video_path_0509.csv", index = False)

# Following steps
1. use the sample frames to find associated plan in QGIS
2. Clean up the attribute prediction results
3. Remove the half person (by compute the w, h ratio)


In [ ]:
videopath = pd.read_csv("../../_data/00_raw/_video_meta/video_path_0509.csv")

In [ ]:

predfiles = os.listdir(predictfolder)
import glob
video_names_done = list(set([x.split("_")[0] for x in predfiles]))
print(f"finished {len(video_names_done) = }")

# check to make sure that all videos are corrected
remain = videopath[~videopath["video_id"].isin(video_names_done)].reset_index(drop = True)
# assert remain.shape[0]==0, "there are videos not processed yet"
print(f"remain {remain.shape[0] = }")

In [ ]:

def merge_mot_human(mot, hu, i):
    motdf = pd.DataFrame(mot[i]["boxes"], columns = ["track_id", "frame_id", "score","x", "y", "w", "h"])
    # motdf["frame_id"] =motdf["frame_id"]+1 
    attr = pd.DataFrame(hu[i], columns = [
        "gender", 
        "age", 
        "side", 
        "glasses",
        "hat",
        "hold_objects_in_front",
        "bag",
        "upper",
        "lower",
        "boots"
    ])
    df = pd.concat([motdf, attr], axis = 1)
    df["frame_id"] = i
    return df

def load_tracking(video_name):
    humanfile = f"{video_name}_human_attr.pickle"
    motfiles = f"{video_name}_mot.pickle"

    human_path = os.path.join(predictfolder, humanfile)
    mot_path = os.path.join(predictfolder, motfiles)
    import pickle
    with open(human_path, "rb") as the_file:
        hu = pickle.load(the_file)
    with open(mot_path, "rb") as the_file:
        mot = pickle.load(the_file)
    DF = []
    for i in hu.keys():
        df = merge_mot_human(mot, hu, i)
        DF.append(df)
    DF = pd.concat(DF).reset_index(drop = True)
    return DF

def find_outliers_IQR(df, field):

   q1=df[field].quantile(0.15)

   q3=df[field].quantile(0.85)

   IQR=q3-q1

   outliers = df[((df[field]<(q1-1.5*IQR)) | (df[field]>(q3+1.5*IQR)))]

   keep = df[((df[field]>=(q1-1.5*IQR)) & (df[field]<=(q3+1.5*IQR)))]

   return outliers, keep

In [ ]:
exportfolder = "../../_data/03_tracking_result/_current_attr_result"
cleaned = os.listdir(exportfolder)
remain_videos = [x for x in video_names_done if x+".csv" not in cleaned]

In [ ]:
remain_videos = ['20100521-115755b04']
DF = load_tracking(remain_videos[0])

In [ ]:
# check the normal w/h ratio
DF['ratio'] = DF['w']/DF['h']
DF['ratio'].describe()

In [ ]:
outliers, keep = find_outliers_IQR(DF, "ratio")

In [ ]:
keep['ratio'].describe()

In [ ]:
for video_name in tqdm(remain_videos):
    if not video_name.endswith(".mp4"):
        try:
            DF = load_tracking(video_name)
            
            DF.to_csv(os.path.join(exportfolder, video_name+".csv"), index = False)
        except:
            # print the error message
            print(f"error in {video_name}")

# Archive below

In [ ]:
# test one video first
testvideo = "20100521-115755b01"
folder = "../../_data/00_raw/videos_current_highres/Downtown Crossing videos (NEW)/VID2/20100521-115755b01.avi"
scenedetect --input B11_G1_Env24_0001.mp4 detect-adaptive list-scenes save-images

In [ ]:
# set up test file
video_folder = "../_data/00_raw/videos_current_highres"
old_video_folder = "../_data/00_raw/_mp4/videos_old_highres"
result_folder = "../_data/03_tracking_result/_current_videos"
files = os.listdir(video_folder)
outfolder = "../_data/05_tracking_result_projected"

if not os.path.exists(outfolder):
    os.makedirs(outfolder)

current_video_sel = [
    "bryant_park/20081008-070830b01.avi",
    "bryant_park/20081008-141944b09.avi",
    "Chestnut Street videos (NEW)/20100519-083343b01.avi",
    "Downtown Crossing videos (NEW)/VID1/"
    
]

In [ ]:
# load image size, fps, and save a sample frame
def get_sample_from_folder(vfolder, minute_start=2, second_start=10):
    files = os.listdir(vfolder)
    for sample_video in tqdm(files):
        videoname = sample_video[:-4]
        video = cv2.VideoCapture(os.path.join(vfolder, sample_video))

        fps = video.get(cv2.CAP_PROP_FPS)
        print("FPS of video ", videoname, ": ", fps)
        size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        print("Size of video ", videoname, ": ", size)
        first_frame = int(fps*(minute_start*60 + second_start))
        video.set(cv2.CAP_PROP_POS_FRAMES, first_frame)
        # save a sample frame for image pairing
        sample_frame_folder = "../_data/02_siteplan/sample_frames"
        ret, frame = video.read()
        cv2.imwrite(os.path.join(sample_frame_folder, f"{videoname}.jpg"), frame)

        video.release()

In [ ]:
get_sample_from_folder(old_video_folder)

In [ ]:
videoname = files[0][:-4]
predpath = os.path.join(result_folder, f'{videoname}.txt')
trace = pd.read_csv(predpath,
                   sep = ',',
                header = None)

# trace.columns = ["frame_id", "track_id", "bbox0", "bbox1", "bbox2", "bbox3","6","7","8","9"]
trace.columns = ["frame_id","track_id","bbox0","bbox1","bbox2","bbox3","score","7","8","9"]
trace["bbox0"] = 720 - trace["bbox0"]
# trace = trace[trace['track_id'].isin(sel)]
trace['loc_x'] = (trace['bbox0'] + trace['bbox0'] + trace['bbox2'])/2
trace['loc_y'] = (trace['bbox1'] + trace['bbox3'])

trace['bbox0'] = trace['bbox0'].astype(int)
trace['bbox1'] = trace['bbox1'].astype(int)
trace['bbox2'] = trace['bbox2'].astype(int)
trace['bbox3'] = trace['bbox3'].astype(int)
trace['frame_id'] = trace['frame_id'].astype(int)

# transform the location to image x, y
trace['y1'] = trace['bbox0']
trace['y2'] = trace['bbox0']+trace['bbox2']

trace['x1'] = trace['bbox1']
trace['x2'] = trace['bbox1']+trace['bbox3']
trace['point'] = trace.apply(lambda row: (int((row['y1']+row['y2'])/2), 
                                                     int((row['x1']+row['x2'])/2)), axis = 1)

In [ ]:
# from 10 second to go, set up original video reading process
# minutes = 7
# seconds = 50
# first_frame = int(fps*(minutes*60 + seconds))
# last_frame = int(fps*(10*60)+0)
# print('frame id =',first_frame)

minutes = 13
seconds = 26
first_frame = int(fps*(minutes*60 + seconds))
last_frame = int(fps*(15*60)+21)
video = cv2.VideoCapture(os.path.join(folder, file_name))
video.set(cv2.CAP_PROP_POS_FRAMES, first_frame)

In [ ]:
# video_out.release()
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_out = cv2.VideoWriter(outfolder + f"/{videoname}_{first_frame}_highres.mp4", fourcc, fps, size)

tracks = trace['track_id'].unique()
colorls = [tuple(random.choices(range(i%255,255), k = 3)) for i in range(len(tracks))]
colorset = dict(zip(tracks, colorls))

In [ ]:
# generate a sample v
# video_out.release()
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# video_out = cv2.VideoWriter(outfolder + f"/{videoname}_{first_frame}_sample.mp4", fourcc, fps, size)
# ret = True
# while ret and frame_id<=last_frame:
#     ret, frame = video.read()
#     frame_id = frame_id+1
#     video_out.write(frame)
# video_out.release()

In [ ]:
ret = True
import numpy as np
frame_id = first_frame
while ret and frame_id<=last_frame:
    ret, frame = video.read()
    frame = cv2.rotate(frame, cv2.ROTATE_180)
    data = trace[trace['frame_id']==frame_id].reset_index(drop = True)
    for j, tr in enumerate(data['track_id'].values):
        cv2.rectangle(frame, 
                          (data.at[j,'bbox0'], 
                           data.at[j,'bbox1']), 
                          (data.at[j,'bbox0']+data.at[j,'bbox2'], 
                           data.at[j,'bbox1']+data.at[j,'bbox3']), 
                              colorset[tr], 1)
        cv2.putText(
                    frame, 
                    str(tr),
                    (data.at[j,'bbox0'], data.at[j,'bbox1']-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, colorset[tr],
                    1,
                    lineType=cv2.LINE_AA
                    )
        # Draw Tracks
#         tracksline = trace[(trace['track_id']==tr)&(trace['frame_id']<=frame_id)].reset_index(drop = True)
#         tracksline.set_index('frame_id', inplace = True)
#         start = max(frame_id-100, 0)
#         for t in range(start, frame_id):
            
#             try:
#             #if tracksline.at[t,'point'] is None or tracksline.at[t-1,'point'] is None: continue
#             #else:
#                 cv2.line(frame, tracksline.at[t,'point'], tracksline.at[t-1,'point'], colorset[tr], 
#                      thickness = int(np.sqrt(49 / float((t-start) + 1)) * 2))
#             except:
#                 continue
    frame_id = frame_id+1
    video_out.write(frame)
video_out.release()

In [ ]:
t = 1800
int(np.sqrt(49 / float((t-start) + 1)) * 2)

In [ ]:
t = 1850
int(np.sqrt(49 / float((t-start) + 1)) * 2)

In [ ]:
int(np.sqrt(100 / float((t-start) + 1)) * 2)

In [ ]:
delta = frame_id - start

In [ ]:
1.5, 0

In [ ]:
t

In [ ]:
int(np.sqrt(49 / float(t + 1)))